In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# Load the dataset
data = pd.read_excel('hechuan ube pre 3.xlsx')  # 确保文件路径正确

# Split the data into features (X) and target variable (Y)
X = data.drop(['ID', 'los', 'elos'], axis=1)
Y = data['elos']

# Convert Y to binary classification (if necessary)
Y = (Y > Y.median()).astype(int)

# Split the data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.1, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Initialize models
rf_model = RandomForestClassifier(random_state=42, n_estimators=100)
rf_model.fit(X_train_scaled, Y_train)

lasso_lr_model = LogisticRegression(penalty='l1', solver='liblinear', C=0.1, random_state=42)
lasso_lr_model.fit(X_train_scaled, Y_train)

xgb_model = XGBClassifier(random_state=42, n_estimators=50, max_depth=3)
xgb_model.fit(X_train_scaled, Y_train)

# Calculate feature importances
# RF
rf_importance = pd.DataFrame({
    'Feature': X.columns,
    'RF_Importance': rf_model.feature_importances_
})

# Lasso Logistic Regression (取绝对值)
lasso_coef = pd.DataFrame({
    'Feature': X.columns,
    'Lasso_Coeff': np.abs(lasso_lr_model.coef_[0])
})

# XGB
xgb_importance = pd.DataFrame({
    'Feature': X.columns,
    'XGB_Importance': xgb_model.feature_importances_
})

# 合并所有重要性
importance_df = rf_importance.merge(lasso_coef, on='Feature').merge(xgb_importance, on='Feature')

# 标准化特征重要性（MinMax归一化）
for col in ['RF_Importance', 'Lasso_Coeff', 'XGB_Importance']:
    importance_df[col] = (importance_df[col] - importance_df[col].min()) / (importance_df[col].max() - importance_df[col].min())

# 计算加权平均（这里假设三个模型权重相同）
importance_df['Ensemble_Importance'] = (
    importance_df['RF_Importance'] + 
    importance_df['Lasso_Coeff'] + 
    importance_df['XGB_Importance']
) / 3

# 取Top 5特征
top_5_features = importance_df.sort_values('Ensemble_Importance', ascending=False).head(5)

# 可视化
plt.figure(figsize=(10, 5))
plt.bar(top_5_features['Feature'], top_5_features['Ensemble_Importance'])
plt.xlabel('Feature')
plt.ylabel('Ensemble Importance Score')
plt.title('Top 5 Features by Ensemble Importance (RF + Lasso LR + XGB)')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

# 打印结果
print("Top 5 Features by Ensemble Importance:")
print(top_5_features[['Feature', 'Ensemble_Importance']])